In [ ]:
import pandas as pd, numpy as np, re, matplotlib.pyplot as plt
import matplotlib as mpl

# ===== File paths =====
csv_path  = "SGF_data.csv"
save_path = "ablation_sgf_2.pdf"

# ===== Style =====
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.50
mpl.rcParams["mathtext.fontset"] = "cm"
mpl.rcParams["mathtext.rm"] = "serif"
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["legend.fontsize"] = 12

LINEWIDTH  = 2.0
MARKERSIZE = 6

# ===== Utilities =====
def parse_pair(val):
    s = str(val).strip()
    if s in {".","-",""}: return None
    s = s.replace("[","").replace("]","")
    parts = [p for p in re.split(r"[, ]+", s) if p]
    if len(parts) >= 2:
        try: return (float(parts[0]), float(parts[1]))
        except: return None
    return None

order_v1 = [(1.0,0.8),(1.0,0.6),(1.0,0.4),(1.0,0.2),(1.0,0.05)]
order_v2 = [(1.0,0.8),(0.8,0.6),(0.6,0.4),(0.4,0.2),(0.2,0.05)]
label_map = {p: f"[{p[0]}, {p[1]}]" for p in set(order_v1+order_v2)}
labels_v1 = [label_map[p] for p in order_v1]
labels_v2 = [label_map[p] for p in order_v2]
wanted_asr = ["Ring-a-bell", "UnlearnDiff", "MMA"]

df = pd.read_csv(csv_path)
df.columns = [c.strip() for c in df.columns]
df["Dataset"] = df["Dataset"].replace({"UnlearnDiff(Sexual)": "UnlearnDiff"})
df["_tw_pair"] = df["Time Windows"].apply(parse_pair)
df["_is_coco"] = df["Dataset"].str.lower().eq("coco")

def prep_asr(order, labels):
    sub = df[(df["_tw_pair"].isin(order)) &
             (df["Dataset"].isin(wanted_asr)) &
             (df["Metric"]=="AttackRate")].copy()
    sub["_tw_label"] = sub["_tw_pair"].map(label_map)
    sub["_tw_label"] = pd.Categorical(sub["_tw_label"], categories=labels, ordered=True)
    return sub.sort_values("_tw_label")

def prep_coco(order, labels):
    sub = df[(df["_tw_pair"].isin(order)) & df["_is_coco"]].copy()
    sub["_tw_label"] = sub["_tw_pair"].map(label_map)
    sub["_tw_label"] = pd.Categorical(sub["_tw_label"], categories=labels, ordered=True)
    sub = sub.sort_values("_tw_label")
    clip = sub[sub["Metric"].str.upper()=="CLIP"][["_tw_label","Value"]].rename(columns={"Value":"CLIP"})
    fid  = sub[sub["Metric"].str.upper()=="FID" ][["_tw_label","Value"]].rename(columns={"Value":"FID"})
    return pd.merge(clip, fid, on="_tw_label", how="outer")

asr_v1, asr_v2 = prep_asr(order_v1, labels_v1), prep_asr(order_v2, labels_v2)
coco_v1, coco_v2 = prep_coco(order_v1, labels_v1), prep_coco(order_v2, labels_v2)

# ===== Y-axis ranges =====
ASR_YLIM  = (0, 0.85)
CLIP_YLIM = (30, 32)
fid_min   = np.nanmin([coco_v1["FID"].min(), coco_v2["FID"].min()])
fid_max   = np.nanmax([coco_v1["FID"].max(), coco_v2["FID"].max()])
FID_YLIM  = (fid_min*0.95, fid_max*1.05)

# ===== 1×4 figure =====
fig, axes = plt.subplots(1, 4, figsize=(18, 3.8), constrained_layout=True)

# (1) ASR — varying λ
ax = axes[0]
for name, g in asr_v1.groupby("Dataset"):
    ax.plot(g["_tw_label"].astype(str), g["Value"], '-o',
            linewidth=LINEWIDTH, markersize=MARKERSIZE,
            markerfacecolor='white', label=name)
ax.set_title("ASR — Varying λ, equalized budget", fontsize=11)
ax.set_xlabel("Time windows"); ax.set_ylabel("ASR ↓")
ax.set_ylim(*ASR_YLIM); ax.grid(True)

order = ["Ring-a-bell", "UnlearnDiff", "MMA"]
handles, labels = ax.get_legend_handles_labels()
ordered = [(h, l) for l, h in sorted(zip(labels, handles), key=lambda x: order.index(x[0]))]
ax.legend(loc="upper left", fontsize=10, frameon=True)

# (2) CoCo-30k — varying λ
axL = axes[1]; axR = axL.twinx()
ln1 = axL.plot(coco_v1["_tw_label"].astype(str), coco_v1["FID"], '-o',
               linewidth=LINEWIDTH, markersize=MARKERSIZE,
               markerfacecolor='white', color="tab:blue", label="FID")
ln2 = axR.plot(coco_v1["_tw_label"].astype(str), coco_v1["CLIP"], '-o',
               linewidth=LINEWIDTH, markersize=MARKERSIZE,
               markerfacecolor='white', color="tab:orange", label="CLIP")
axL.set_title("CoCo-30k — Varying λ, equalized budget", fontsize=11)
axL.set_xlabel("Time windows"); axL.set_ylabel("FID ↓"); axR.set_ylabel("CLIP ↑")
axL.set_ylim(*FID_YLIM); axR.set_ylim(*CLIP_YLIM); axL.grid(True)
lines = ln1+ln2; labels = [l.get_label() for l in lines]
axL.legend(lines, labels, fontsize=10, frameon=True, loc="upper left")

# (3) ASR — fixed λ
ax = axes[2]
for name, g in asr_v2.groupby("Dataset"):
    ax.plot(g["_tw_label"].astype(str), g["Value"], '-o',
            linewidth=LINEWIDTH, markersize=MARKERSIZE,
            markerfacecolor='white', label=name)
ax.set_title("ASR — Fixed λ, varying interval lengths", fontsize=11)
ax.set_xlabel("Time windows"); ax.set_ylabel("ASR ↓")
ax.set_ylim(*ASR_YLIM); ax.grid(True)

order = ["Ring-a-bell", "UnlearnDiff", "MMA"]
handles, labels = ax.get_legend_handles_labels()
ordered = [(h, l) for l, h in sorted(zip(labels, handles), key=lambda x: order.index(x[0]))]
ax.legend(loc="upper left", fontsize=10, frameon=True)

# (4) CoCo-30k — fixed λ
axL = axes[3]; axR = axL.twinx()
ln1 = axL.plot(coco_v2["_tw_label"].astype(str), coco_v2["FID"], '-o',
               linewidth=LINEWIDTH, markersize=MARKERSIZE,
               markerfacecolor='white', color="tab:blue", label="FID")
ln2 = axR.plot(coco_v2["_tw_label"].astype(str), coco_v2["CLIP"], '-o',
               linewidth=LINEWIDTH, markersize=MARKERSIZE,
               markerfacecolor='white', color="tab:orange", label="CLIP")
axL.set_title("CoCo-30k — Fixed λ, varying interval lengths", fontsize=11)
axL.set_xlabel("Time windows"); axL.set_ylabel("FID ↓"); axR.set_ylabel("CLIP ↑")
axL.set_ylim(*FID_YLIM); axR.set_ylim(*CLIP_YLIM); axL.grid(True)
lines = ln1+ln2; labels = [l.get_label() for l in lines]
axL.legend(lines, labels, fontsize=10, frameon=True, loc="upper left")

plt.savefig(save_path, dpi=200, bbox_inches="tight")
plt.close()
print(f"Saved: {save_path}")

Saved: ablation_sgf_2.pdf
